In [1]:
#use spacy to preprocess the original text for entity extraction
from __future__ import unicode_literals
from textacy import fileio, preprocess, extract, Corpus 

In [2]:
from textacy import Doc
#sample_corpus = corpus_train.docs[110]
sample_doc = Doc("Two buses belong to SMRT ( bus)")
preprocess.replace_numbers(sample_doc.text)

vob_sub = sample_doc.spacy_stringstore
sample_vob = ['one', 'Two', 'two', 'SMRT', 'smrt', 'bus', 'buses']
print(map(lambda i:vob_sub[i], sample_vob))

print(vob_sub.size)

print(map(lambda i:vob_sub[i.text], sample_doc))
print(map(lambda i:i.lemma, sample_doc))

[519, 2422, 659, 207883, 119646, 2637, 10134]
1510241
[2422, 10134, 4382, 468, 207883, 506, 2637, 487]
[659, 2637, 4382, 468, 119646, 506, 2637, 487]


In [3]:
summary_docs = fileio.read_file_lines('/home/SMRT-labeled/summary-docs')
raw_text = [t for t in summary_docs]
prep_text = [preprocess.replace_numbers(t, '_NUM_') for t in raw_text]

corpus_train = Corpus('en', texts = prep_text)

In [4]:
corpus_train.n_docs

1766

In [5]:
#build the dictionary
from collections import Counter

dict_all = Counter(map(lambda i:i.lemma_, corpus_train.docs[0]))

for doc_i in corpus_train.docs[1:]:
    dict_all = dict_all + Counter(map(lambda i:i.lemma_, doc_i))

In [6]:
import json

dict_all.pop('\n', None)

dict_all_cut = {k:v for k, v in dict_all.iteritems() if v>=5}

with open('./dict/vocab_freq.json', 'w') as fp:
    json.dump(dict_all_cut, fp)
    
dict_key = [k for k,v in dict_all_cut.iteritems()]
dict_key_v = {k:v for k, v in zip(dict_key, range(len(dict_key)))}

with open('./dict/vocab_list.json', 'w') as fp:
    json.dump(dict_key_v, fp)

In [7]:
len(dict_all_cut.keys())

2632

In [ ]:
sent_len = []

for doc_i in corpus_train.docs:
    for sent_i in doc_i.sents:
        sent_len.append(len(sent_i))

In [ ]:
example_sent = [i for i in corpus_train.docs[2].sents]
print(example_sent[1])
print('Length: '+str(len(example_sent[1])))

In [ ]:
#get the rough estimation of sentence length
print(max(sent_len))
print(min(sent_len))

In [ ]:
from matplotlib import pyplot
%matplotlib inline

#hist(token_size)
pyplot.hist(sent_len)
pyplot.show()

In [ ]:
import csv
import random

random.sample(range(100), 10)

In [ ]:
with open("/home/NER-label/training-set-to-label", 'wb') as f:
    csv_w = csv.writer(f, quotechar = str("'"))
    
    train_text = random.sample(range(corpus_train.n_docs), 1000)
    
    for corp_i in train_text:
        corp = corpus_train.docs[corp_i]
        ne_tag_list = ['O'] * corp.n_tokens

        ne = extract.named_entities(corp)
        for i_ne in ne:
            ne_tag_list[i_ne.start:i_ne.end] = [i_ne.label_]*(i_ne.end - i_ne.start)

        pos_tag_list = corp.pos_tagged_text

        sent_len = [0]
        for pos_sent in pos_tag_list:
            sent_len.append(len(pos_sent))

        new_ne_tag_list = [ne_tag_list[(reduce(lambda x, y:(x+y), sent_len[0:i])):(reduce(lambda x, y:(x+y), sent_len[0:i])+sent_len[i])] for i in range(1, len(sent_len)) ]

        for i_sent in range(len(pos_tag_list)):
            combined = (zip(map(lambda x:x[1], pos_tag_list[i_sent]), map(lambda x:x[0], pos_tag_list[i_sent]), new_ne_tag_list[i_sent]))
            for v_i in combined:
                if(v_i[1] != u'\n'):
                    csv_w.writerow(v_i)
            csv_w.writerow([""]*3)